## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [72]:
# 필요한 라이브러리를 설치합니다.
!pip install timm

In [73]:
!pip install transformers

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [74]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [75]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [77]:
class ImageDataset(Dataset):
    def __init__(self, csv_files, img_paths, transform=None):
        # csv_files와 img_paths를 리스트로 처리할 수 있게 만들기
        if isinstance(csv_files, str):
            csv_files = [csv_files]
        if isinstance(img_paths, str):
            img_paths = [img_paths]

        # 여러 CSV 파일들을 하나의 DataFrame으로 결합
        self.df = pd.concat([pd.read_csv(csv) for csv in csv_files], ignore_index=True)
        self.img_paths = img_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 데이터프레임에서 이미지 이름(ID)과 타겟 값을 가져오기
        img_name, target = self.df.iloc[idx]['ID'], self.df.iloc[idx]['target']
        img = None
        # target 값을 정수형으로 변환
        target = int(target)  # target 값을 정수형으로 변환합니다.

        # 이미지 경로 리스트를 순회하며 이미지 파일을 찾기
        for path in self.img_paths:
            img_path = os.path.join(path, img_name)
            if os.path.exists(img_path):
                img = Image.open(img_path).convert('RGB')
                # Move transform usage here
                inputs = self.transform(images=img, return_tensors="pt")
                input_pixels = inputs['pixel_values'].squeeze()  # Remove batch dimension
                break


        return input_pixels, torch.tensor(target, dtype=torch.long)


## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [78]:
for model in timm.list_models():
    print(model)

bat_resnext26ts
beit_base_patch16_224
beit_base_patch16_384
beit_large_patch16_224
beit_large_patch16_384
beit_large_patch16_512
beitv2_base_patch16_224
beitv2_large_patch16_224
botnet26t_256
botnet50ts_256
caformer_b36
caformer_m36
caformer_s18
caformer_s36
cait_m36_384
cait_m48_448
cait_s24_224
cait_s24_384
cait_s36_384
cait_xs24_384
cait_xxs24_224
cait_xxs24_384
cait_xxs36_224
cait_xxs36_384
coat_lite_medium
coat_lite_medium_384
coat_lite_mini
coat_lite_small
coat_lite_tiny
coat_mini
coat_small
coat_tiny
coatnet_0_224
coatnet_0_rw_224
coatnet_1_224
coatnet_1_rw_224
coatnet_2_224
coatnet_2_rw_224
coatnet_3_224
coatnet_3_rw_224
coatnet_4_224
coatnet_5_224
coatnet_bn_0_rw_224
coatnet_nano_cc_224
coatnet_nano_rw_224
coatnet_pico_rw_224
coatnet_rmlp_0_rw_224
coatnet_rmlp_1_rw2_224
coatnet_rmlp_1_rw_224
coatnet_rmlp_2_rw_224
coatnet_rmlp_2_rw_384
coatnet_rmlp_3_rw_224
coatnet_rmlp_nano_rw_224
coatnext_nano_rw_224
convformer_b36
convformer_m36
convformer_s18
convformer_s36
convit_base
conv

In [79]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = 'data/'

# model config
model_name = 'efficientnet_b0' # 'resnet34' # 'resnet50' 'wide_resnet101_2', ... # efficientnet_b0

# training config
img_size = 256 # 256
LR = 1e-3
EPOCHS = 200
BATCH_SIZE = 32
num_workers = 0

## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [80]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [81]:
## ALL : 0.9382 / A_13 제외 : 0.9245
from transformers import AutoModelForImageClassification, AutoImageProcessor

trn_transform = AutoImageProcessor.from_pretrained("microsoft/dit-large-finetuned-rvlcdip")
tst_transform = AutoImageProcessor.from_pretrained("microsoft/dit-large-finetuned-rvlcdip")
model = AutoModelForImageClassification.from_pretrained("microsoft/dit-large-finetuned-rvlcdip")

## ALL : 0.9382
# trn_transform = AutoImageProcessor.from_pretrained("microsoft/dit-base-finetuned-rvlcdip")
# tst_transform  = AutoImageProcessor.from_pretrained("microsoft/dit-base-finetuned-rvlcdip")
# model = AutoModelForImageClassification.from_pretrained("microsoft/dit-base-finetuned-rvlcdip")

preprocessor_config.json:   0%|          | 0.00/302 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
/opt/conda/lib/python3.10/site-packages/transformers/models/beit/image_processing_beit.py:110: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


config.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

In [82]:
# CSV 파일과 해당 이미지 경로
csv_list = [data_path + "/A_train.csv", data_path + "/train.csv", data_path + "/A_13_train.csv", data_path + "/rotate_train.csv", data_path + "/rotateflip_train.csv", data_path + "/A_rotateandflip_train.csv"]
path_list = [data_path + "/augmented/A", data_path + "/train", data_path + "/augmented/A_13", data_path + "/augmented/rotate", data_path + "/augmented/rotateflip", data_path + "/augmented/rotateflip"]

# 훈련 데이터셋 정의
trn_dataset = ImageDataset(
    csv_files=csv_list,
    img_paths=path_list,
    transform=trn_transform
)

# 테스트 데이터셋 정의 (단일 CSV 파일과 이미지 경로)
tst_dataset = ImageDataset(
    csv_files=data_path + "/sample_submission.csv",
    img_paths=[data_path + "/test"], # 단일 경로도 리스트로 감싸기
    transform=tst_transform
)

print(len(trn_dataset), len(tst_dataset))


66640 3140


In [84]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

In [85]:
# # split
# from torch.utils.data import DataLoader, random_split

# # 전체 훈련 데이터셋 크기
# total_size = len(trn_dataset)

# # 훈련 및 검증 세트의 크기를 정의
# train_size = int(total_size * 0.8)  # 전체 데이터의 80%를 훈련용으로 사용
# valid_size = total_size - train_size  # 나머지 20%를 검증용으로 사용

# # 데이터셋을 훈련 및 검증 세트로 나눔
# train_dataset, valid_dataset = random_split(trn_dataset, [train_size, valid_size])

# # 훈련 및 검증 DataLoader 정의
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
# valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

# print(train_size, valid_size)


53312 13328


## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [86]:
# # load model
# model = timm.create_model(
#     model_name,
#     pretrained=True,
#     num_classes=17
# ).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

In [87]:
# 검증 과정을 위한 함수
def validate(loader, model, loss_fn, device):
    model.eval()  # 모델을 평가 모드로 설정
    val_loss = 0
    preds_list = []
    targets_list = []
    
    with torch.no_grad():  # 기울기 계산 비활성화
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)
            
            preds = model(image)
            loss = loss_fn(preds, targets)
            
            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())
    
    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')
    
    ret = {
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
    }
    
    return ret

In [88]:
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)


        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret


In [90]:
import torch
from torch.utils.data import DataLoader, Dataset, Subset
from torch import nn, optim
from torchvision import transforms
import numpy as np
from sklearn.model_selection import KFold
import timm
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Early stopping을 위한 클래스 초기화
        patience: 성능 개선이 없는 에폭 수
        verbose: 메시지 출력 여부
        delta: 성능 개선으로 간주되기 위한 최소 변화량
        path: 모델 저장 경로
        trace_func: 로그 출력 함수
        """
        
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """
        검증 손실이 감소하면 모델을 저장
        """
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

# 데이터셋 준비
dataset = trn_dataset

# K-Fold Cross-Validation 설정
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)

# 결과를 저장할 리스트 초기화
fold_results = []

# K-Fold Cross-Validation 시작
for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    # DataLoader 정의
    train_subsampler = Subset(dataset, train_ids)
    valid_subsampler = Subset(dataset, valid_ids)
    train_loader = DataLoader(train_subsampler, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_subsampler, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
    
    # 모델 초기화
    model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
    
    # 손실 함수와 최적화 방법 정의
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LR)
    
    # Early Stopping 초기화
    early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.001, path=f'fold_{fold}_checkpoint.pt')
    
    # 폴드별 훈련 및 검증 성능 기록
    fold_performance = {
        'train_acc': [],
        'train_f1': [],
        'val_acc': [],
        'val_f1': []
    }
    
    # 폴드별 훈련 및 검증
    for epoch in range(EPOCHS):
        # 훈련
        train_results = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
        fold_performance['train_acc'].append(train_results['train_acc'])
        fold_performance['train_f1'].append(train_results['train_f1'])
        
        # 검증
        val_results = validate(valid_loader, model, loss_fn, device)
        fold_performance['val_acc'].append(val_results['val_acc'])
        fold_performance['val_f1'].append(val_results['val_f1'])
        
        early_stopping(val_results['val_loss'], model)
        if early_stopping.early_stop:
            print("Early stopping")
            break
    
    model.load_state_dict(torch.load(f'fold_{fold}_checkpoint.pt'))
    
    fold_results.append(fold_performance)
    print(f'Fold {fold} completed. Best val accuracy: {val_results["val_acc"]}, Best val F1 Score: {val_results["val_f1"]}\n')

# 최종 성능 계산 및 출력
train_accs = [np.mean(fold['train_acc']) for fold in fold_results]
train_f1s = [np.mean(fold['train_f1']) for fold in fold_results]
val_accs = [np.mean(fold['val_acc']) for fold in fold_results]
val_f1s = [np.mean(fold['val_f1']) for fold in fold_results]

print(f'Average Train Accuracy: {np.mean(train_accs)}, Average Train F1 Score: {np.mean(train_f1s)}')
print(f'Average Validation Accuracy: {np.mean(val_accs)}, Average Validation F1 Score: {np.mean(val_f1s)}')


FOLD 0
--------------------------------


  0%|          | 0/1666 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3322b12b90>Exception ignored in: 
Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f3322b12b90>
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
Traceback (most recent call last):
      File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
self._shutdown_workers()    
Exception ignored in: self._shutdown_workers()  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f3322b12b90>

      File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
if w.is_alive():Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    if w.is_

Validation loss decreased (inf --> 0.134892).  Saving model ...


  0%|          | 0/1666 [00:00<?, ?it/s]

...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3322b12b90>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3322b12b90>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/op

EarlyStopping counter: 4 out of 5


  0%|          | 0/1666 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3322b12b90>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3322b12b90><function _MultiProcessingDataLoaderIter.__del__ at 0x7f3322b12b90>

Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
  File "/opt/conda/lib/python3.10/site-packages/t

EarlyStopping counter: 5 out of 5
Early stopping
Fold 4 completed. Best val accuracy: 0.993922569027611, Best val F1 Score: 0.9936832509246777

Average Train Accuracy: 0.9829714042933541, Average Train F1 Score: 0.9825553762445424
Average Validation Accuracy: 0.9852166892511434, Average Validation F1 Score: 0.9848147550464936


# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [96]:
import numpy as np
from scipy.stats import mode
from tqdm.auto import tqdm

# 테스트 데이터셋에 대한 각 폴드의 모델 예측 수행
k_folds = 5

predictions = []
for fold in range(k_folds):
    # 각 폴드의 모델을 로드합니다.
    model.load_state_dict(torch.load(f'fold_{fold}_checkpoint.pt'))
    model.eval()
    fold_predictions = []
    with torch.no_grad():
        for images, _ in tqdm(tst_loader, desc=f'Predicting with fold {fold}'):
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            fold_predictions.extend(predicted.cpu().numpy())
    predictions.append(fold_predictions)

# 앙상블 예측: 다수결 보팅
predictions = np.array(predictions)
predictions = np.transpose(predictions, (1, 0))

preds_list = []
for i in range(len(predictions)):
    mode_result = mode(predictions[i])
    preds_list.append(mode_result[0])
    
len(preds_list)

Predicting with fold 0:   0%|          | 0/99 [00:00<?, ?it/s]

Predicting with fold 1:   0%|          | 0/99 [00:00<?, ?it/s]

Predicting with fold 2:   0%|          | 0/99 [00:00<?, ?it/s]

Predicting with fold 3:   0%|          | 0/99 [00:00<?, ?it/s]

Predicting with fold 4:   0%|          | 0/99 [00:00<?, ?it/s]

3140

In [98]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [99]:
sample_submission_df = pd.read_csv(data_path + "/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [100]:
pred_df.to_csv("kfoldpreds_0215.csv", index=False)

In [101]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,12
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,12
4,00901f504008d884.jpg,2
